In [5]:
# PYTHON IMPORTS
import datetime
from dateutil import relativedelta
# THIRD PARTY IMPORTS
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb

In [6]:
DEPENDENT_COLUMN = 'sales'

PARAMS = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [7]:
# Read data
df = pd.read_csv(
    'data/sintomasCovid.csv',
    usecols=['week', 'sintomas_covid', 'sales']
)
df.week = pd.to_datetime(df.week)
df.set_index('week', inplace=True)

In [15]:
x = df[['sintomas_covid']]
y = df[['sales']]
train_date = datetime.date(2022, 1, 9)
test_date = train_date + relativedelta.relativedelta(weeks=1)

In [16]:
train_x = x[:train_date]
train_y = y[:train_date][[DEPENDENT_COLUMN]]
test_x = x[test_date:]
test_y = y[test_date:][[DEPENDENT_COLUMN]]

In [20]:
lgb_train = lgb.Dataset(train_x, train_y)
gbm = lgb.train(PARAMS, lgb_train, num_boost_round=500)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000056 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

In [26]:
df_test = df[test_date:]

In [28]:
df_test["preds"] = gbm.predict(
                    test_x[test_date:],
                    num_iteration=gbm.best_iteration
                )

c:\Users\Sebastian\Documents\VDC\MVTF\forecasting\.venv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
df_test

,sintomas_covid,sales,preds
week,,,
2022-01-16,70,726,1120.822184
2022-01-23,50,811,1052.508777
2022-01-30,34,995,1072.401049
2022-02-06,22,1130,1322.365945
2022-02-13,14,1169,1271.748462
2022-02-20,11,1248,1271.748462
2022-02-27,10,1202,1271.748462
2022-03-06,7,1312,1271.748462
2022-03-13,8,1284,1271.748462
